
# **Compuational Drug Discovery - First Notebook**


## **ChEMBL Database**

The [*ChEMBL Database*](https://www.ebi.ac.uk/chembl/) is a database that contains curated bioactivity data of more than 2 million compounds. It is compiled from more than 76,000 documents, 1.2 million assays and the data spans 13,000 targets and 1,800 cells and 33,000 indications.
[Data as of March 25, 2020; ChEMBL version 26].

## **Importing libraries**

In [1]:
# Import necessary libraries
import pandas as pd
from chembl_webresource_client.new_client import new_client

## **Search for Target protein**

### **Target search for Enoyl Acyl Carrier Protein Reductase code name ('CHEMBL1849')**

In [32]:
# Target search for coronavirus
target = new_client.target
target_query = target.search('CHEMBL1849')
targets = pd.DataFrame.from_dict(target_query)
targets

,cross_references,organism,pref_name,score,species_group_flag,target_chembl_id,target_components,target_type,tax_id
0,"[{'xref_id': 'P0A5Y6', 'xref_name': None, 'xre...",Mycobacterium tuberculosis,Enoyl-[acyl-carrier-protein] reductase,17.0,False,CHEMBL1849,"[{'accession': 'P9WGR1', 'component_descriptio...",SINGLE PROTEIN,1773


In [33]:
selected_target = targets.target_chembl_id[0]
selected_target

'CHEMBL1849'

In [34]:
try:
    activity = new_client.activity
    # Use slice notation instead of limit
    res = activity.filter(target_chembl_id=selected_target).filter(standard_type="IC50")[0:10]
    
    # Print the query to check what we're requesting
    print(f"Target: {selected_target}")
    
    # Try creating the DataFrame with explicit list conversion
    df = pd.DataFrame(list(res))
    print("DataFrame shape:", df.shape)
    
except Exception as e:
    print(f"Error occurred: {str(e)}")

Target: CHEMBL1849
DataFrame shape: (10, 46)


In [29]:
# Look at the column names first
print("Columns in the dataset:")
print(df.columns.tolist())

# Show a summary of the key columns
print("\nFirst few rows of key measurements:")
print(df[['molecule_chembl_id', 'standard_type', 'standard_value', 'standard_units']].head())

# Check for any missing values
print("\nMissing values in key columns:")
print(df[['standard_value', 'standard_units']].isnull().sum())

Columns in the dataset:
['action_type', 'activity_comment', 'activity_id', 'activity_properties', 'assay_chembl_id', 'assay_description', 'assay_type', 'assay_variant_accession', 'assay_variant_mutation', 'bao_endpoint', 'bao_format', 'bao_label', 'canonical_smiles', 'data_validity_comment', 'data_validity_description', 'document_chembl_id', 'document_journal', 'document_year', 'ligand_efficiency', 'molecule_chembl_id', 'molecule_pref_name', 'parent_molecule_chembl_id', 'pchembl_value', 'potential_duplicate', 'qudt_units', 'record_id', 'relation', 'src_id', 'standard_flag', 'standard_relation', 'standard_text_value', 'standard_type', 'standard_units', 'standard_upper_value', 'standard_value', 'target_chembl_id', 'target_organism', 'target_pref_name', 'target_tax_id', 'text_value', 'toid', 'type', 'units', 'uo_units', 'upper_value', 'value']

First few rows of key measurements:
  molecule_chembl_id standard_type standard_value standard_units
0       CHEMBL133897          IC50          7

Here, we will retrieve only bioactivity data for *Enoyl Acyl Carrier Protein Reductase* that are reported as pChEMBL values.

In [38]:
activity = new_client.activity
res = activity.filter(target_chembl_id=selected_target).filter(standard_type="IC50")

In [42]:
import time

try:
    activity = new_client.activity
    all_results = []
    
    # Get data in smaller chunks (5 at a time) and use a longer delay
    for i in range(0, 1000, 5):  # Get up to 1000 records, 5 at a time
        chunk = activity.filter(target_chembl_id=selected_target).filter(standard_type="IC50")[i:i+5]
        try:
            results = list(chunk)
            if not results:  # If no more results, break
                break
            all_results.extend(results)
            time.sleep(2)  # Increase delay to 2 seconds between requests
            print(f"Successfully retrieved chunk {i//5 + 1}, total records: {len(all_results)}")
        except Exception as e:
            print(f"Error on chunk {i}: {str(e)}")
            time.sleep(5)  # Longer delay after an error
            continue
            
    # Create DataFrame from all collected results
    if all_results:
        df = pd.DataFrame(all_results)
        print(f"\nFinal DataFrame shape: {df.shape}")
        # Save the data to avoid having to request it again
        df.to_csv('chembl_activity_data.csv', index=False)
    else:
        print("No results were retrieved successfully")

except Exception as e:
    print(f"Error occurred: {str(e)}")

Successfully retrieved chunk 1, total records: 5
Successfully retrieved chunk 2, total records: 10
Successfully retrieved chunk 3, total records: 15
Successfully retrieved chunk 4, total records: 20
Successfully retrieved chunk 5, total records: 25
Successfully retrieved chunk 6, total records: 30
Successfully retrieved chunk 7, total records: 35
Successfully retrieved chunk 8, total records: 40
Successfully retrieved chunk 9, total records: 45
Successfully retrieved chunk 10, total records: 50
Successfully retrieved chunk 11, total records: 55
Successfully retrieved chunk 12, total records: 60
Successfully retrieved chunk 13, total records: 65
Successfully retrieved chunk 14, total records: 70
Successfully retrieved chunk 15, total records: 75
Successfully retrieved chunk 16, total records: 80
Successfully retrieved chunk 17, total records: 85
Successfully retrieved chunk 18, total records: 90
Successfully retrieved chunk 19, total records: 95
Successfully retrieved chunk 20, total re

In [43]:
df = pd.DataFrame.from_dict(res)

In [44]:
df

,action_type,activity_comment,activity_id,activity_properties,assay_chembl_id,assay_description,assay_type,assay_variant_accession,assay_variant_mutation,bao_endpoint,...,target_organism,target_pref_name,target_tax_id,text_value,toid,type,units,uo_units,upper_value,value
0,None,None,1774267,[],CHEMBL907779,Inhibition of Mycobacterium tuberculosis InhA,B,None,None,BAO_0000190,...,Mycobacterium tuberculosis,Enoyl-[acyl-carrier-protein] reductase,1773,None,None,IC50,uM,UO_0000065,None,10.66
1,None,None,1774268,[],CHEMBL907779,Inhibition of Mycobacterium tuberculosis InhA,B,None,None,BAO_0000190,...,Mycobacterium tuberculosis,Enoyl-[acyl-carrier-protein] reductase,1773,None,None,IC50,uM,UO_0000065,None,100.0
2,None,None,1774269,[],CHEMBL907779,Inhibition of Mycobacterium tuberculosis InhA,B,None,None,BAO_0000190,...,Mycobacterium tuberculosis,Enoyl-[acyl-carrier-protein] reductase,1773,None,None,IC50,uM,UO_0000065,None,100.0
3,None,None,1774270,[],CHEMBL907779,Inhibition of Mycobacterium tuberculosis InhA,B,None,None,BAO_0000190,...,Mycobacterium tuberculosis,Enoyl-[acyl-carrier-protein] reductase,1773,None,None,IC50,uM,UO_0000065,None,100.0
4,None,None,1774271,[],CHEMBL907779,Inhibition of Mycobacterium tuberculosis InhA,B,None,None,BAO_0000190,...,Mycobacterium tuberculosis,Enoyl-[acyl-carrier-protein] reductase,1773,None,None,IC50,uM,UO_0000065,None,100.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
419,"{'action_type': 'INHIBITOR', 'description': 'N...",None,24994756,[],CHEMBL5231331,Inhibition of Mycobacterial tuberculosis ATCC ...,B,None,None,BAO_0000190,...,Mycobacterium tuberculosis,Enoyl-[acyl-carrier-protein] reductase,1773,None,None,IC50,ug ml-1,UO_0000274,None,4.5
420,"{'action_type': 'INHIBITOR', 'description': 'N...",None,24994807,[],CHEMBL5231347,Inhibition of Mycobacterium tuberculosis ATCC ...,B,None,None,BAO_0000190,...,Mycobacterium tuberculosis,Enoyl-[acyl-carrier-protein] reductase,1773,None,None,IC50,nM,UO_0000065,None,570.0
421,"{'action_type': 'INHIBITOR', 'description': 'N...",None,24994808,[],CHEMBL5231347,Inhibition of Mycobacterium tuberculosis ATCC ...,B,None,None,BAO_0000190,...,Mycobacterium tuberculosis,Enoyl-[acyl-carrier-protein] reductase,1773,None,None,IC50,nM,UO_0000065,None,4.0
422,"{'action_type': 'INHIBITOR', 'description': 'N...",None,24994842,[],CHEMBL5231368,Inhibition of Mycobacterium tuberculosis InhA ...,B,None,None,BAO_0000190,...,Mycobacterium tuberculosis,Enoyl-[acyl-carrier-protein] reductase,1773,None,None,IC50,uM,UO_0000065,None,4.7


In [45]:
df.columns.values

array(['action_type', 'activity_comment', 'activity_id',
       'activity_properties', 'assay_chembl_id', 'assay_description',
       'assay_type', 'assay_variant_accession', 'assay_variant_mutation',
       'bao_endpoint', 'bao_format', 'bao_label', 'canonical_smiles',
       'data_validity_comment', 'data_validity_description',
       'document_chembl_id', 'document_journal', 'document_year',
       'ligand_efficiency', 'molecule_chembl_id', 'molecule_pref_name',
       'parent_molecule_chembl_id', 'pchembl_value',
       'potential_duplicate', 'qudt_units', 'record_id', 'relation',
       'src_id', 'standard_flag', 'standard_relation',
       'standard_text_value', 'standard_type', 'standard_units',
       'standard_upper_value', 'standard_value', 'target_chembl_id',
       'target_organism', 'target_pref_name', 'target_tax_id',
       'text_value', 'toid', 'type', 'units', 'uo_units', 'upper_value',
       'value'], dtype=object)

Finally we will save the resulting bioactivity data to a CSV file **bioactivity_data.csv**.

In [50]:
df.to_csv('enoyl_acyl_carrier_protein_reductase_01_bioactivity_data_raw.csv', index=False)

## **Handling missing data**
If any compounds has missing value for the **standard_value** and **canonical_smiles** column then drop it.

In [51]:
df2 = df[df.standard_value.notna()]
df2 = df2[df.canonical_smiles.notna()]
df2

C:\Users\Promise Sunday\AppData\Local\Temp\ipykernel_11016\3852201246.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df2 = df2[df.canonical_smiles.notna()]


,action_type,activity_comment,activity_id,activity_properties,assay_chembl_id,assay_description,assay_type,assay_variant_accession,assay_variant_mutation,bao_endpoint,...,target_organism,target_pref_name,target_tax_id,text_value,toid,type,units,uo_units,upper_value,value
0,None,None,1774267,[],CHEMBL907779,Inhibition of Mycobacterium tuberculosis InhA,B,None,None,BAO_0000190,...,Mycobacterium tuberculosis,Enoyl-[acyl-carrier-protein] reductase,1773,None,None,IC50,uM,UO_0000065,None,10.66
1,None,None,1774268,[],CHEMBL907779,Inhibition of Mycobacterium tuberculosis InhA,B,None,None,BAO_0000190,...,Mycobacterium tuberculosis,Enoyl-[acyl-carrier-protein] reductase,1773,None,None,IC50,uM,UO_0000065,None,100.0
2,None,None,1774269,[],CHEMBL907779,Inhibition of Mycobacterium tuberculosis InhA,B,None,None,BAO_0000190,...,Mycobacterium tuberculosis,Enoyl-[acyl-carrier-protein] reductase,1773,None,None,IC50,uM,UO_0000065,None,100.0
3,None,None,1774270,[],CHEMBL907779,Inhibition of Mycobacterium tuberculosis InhA,B,None,None,BAO_0000190,...,Mycobacterium tuberculosis,Enoyl-[acyl-carrier-protein] reductase,1773,None,None,IC50,uM,UO_0000065,None,100.0
4,None,None,1774271,[],CHEMBL907779,Inhibition of Mycobacterium tuberculosis InhA,B,None,None,BAO_0000190,...,Mycobacterium tuberculosis,Enoyl-[acyl-carrier-protein] reductase,1773,None,None,IC50,uM,UO_0000065,None,100.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
419,"{'action_type': 'INHIBITOR', 'description': 'N...",None,24994756,[],CHEMBL5231331,Inhibition of Mycobacterial tuberculosis ATCC ...,B,None,None,BAO_0000190,...,Mycobacterium tuberculosis,Enoyl-[acyl-carrier-protein] reductase,1773,None,None,IC50,ug ml-1,UO_0000274,None,4.5
420,"{'action_type': 'INHIBITOR', 'description': 'N...",None,24994807,[],CHEMBL5231347,Inhibition of Mycobacterium tuberculosis ATCC ...,B,None,None,BAO_0000190,...,Mycobacterium tuberculosis,Enoyl-[acyl-carrier-protein] reductase,1773,None,None,IC50,nM,UO_0000065,None,570.0
421,"{'action_type': 'INHIBITOR', 'description': 'N...",None,24994808,[],CHEMBL5231347,Inhibition of Mycobacterium tuberculosis ATCC ...,B,None,None,BAO_0000190,...,Mycobacterium tuberculosis,Enoyl-[acyl-carrier-protein] reductase,1773,None,None,IC50,nM,UO_0000065,None,4.0
422,"{'action_type': 'INHIBITOR', 'description': 'N...",None,24994842,[],CHEMBL5231368,Inhibition of Mycobacterium tuberculosis InhA ...,B,None,None,BAO_0000190,...,Mycobacterium tuberculosis,Enoyl-[acyl-carrier-protein] reductase,1773,None,None,IC50,uM,UO_0000065,None,4.7


In [52]:
len(df2.canonical_smiles.unique())

340

In [53]:
df2_nr = df2.drop_duplicates(['canonical_smiles'])
df2_nr

,action_type,activity_comment,activity_id,activity_properties,assay_chembl_id,assay_description,assay_type,assay_variant_accession,assay_variant_mutation,bao_endpoint,...,target_organism,target_pref_name,target_tax_id,text_value,toid,type,units,uo_units,upper_value,value
0,None,None,1774267,[],CHEMBL907779,Inhibition of Mycobacterium tuberculosis InhA,B,None,None,BAO_0000190,...,Mycobacterium tuberculosis,Enoyl-[acyl-carrier-protein] reductase,1773,None,None,IC50,uM,UO_0000065,None,10.66
1,None,None,1774268,[],CHEMBL907779,Inhibition of Mycobacterium tuberculosis InhA,B,None,None,BAO_0000190,...,Mycobacterium tuberculosis,Enoyl-[acyl-carrier-protein] reductase,1773,None,None,IC50,uM,UO_0000065,None,100.0
2,None,None,1774269,[],CHEMBL907779,Inhibition of Mycobacterium tuberculosis InhA,B,None,None,BAO_0000190,...,Mycobacterium tuberculosis,Enoyl-[acyl-carrier-protein] reductase,1773,None,None,IC50,uM,UO_0000065,None,100.0
3,None,None,1774270,[],CHEMBL907779,Inhibition of Mycobacterium tuberculosis InhA,B,None,None,BAO_0000190,...,Mycobacterium tuberculosis,Enoyl-[acyl-carrier-protein] reductase,1773,None,None,IC50,uM,UO_0000065,None,100.0
4,None,None,1774271,[],CHEMBL907779,Inhibition of Mycobacterium tuberculosis InhA,B,None,None,BAO_0000190,...,Mycobacterium tuberculosis,Enoyl-[acyl-carrier-protein] reductase,1773,None,None,IC50,uM,UO_0000065,None,100.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
412,"{'action_type': 'INHIBITOR', 'description': 'N...",None,22969102,[],CHEMBL4770222,Inhibition of recombinant Mycobacterium tuberc...,B,None,None,BAO_0000190,...,Mycobacterium tuberculosis,Enoyl-[acyl-carrier-protein] reductase,1773,None,None,IC50,uM,UO_0000065,None,15.6
413,"{'action_type': 'INHIBITOR', 'description': 'N...",None,24689267,"[{'comments': None, 'relation': '=', 'result_f...",CHEMBL5105160,Inhibition of His-tagged Mycobacterium tubercu...,B,None,None,BAO_0000190,...,Mycobacterium tuberculosis,Enoyl-[acyl-carrier-protein] reductase,1773,None,None,IC50,uM,UO_0000065,None,0.03
414,"{'action_type': 'INHIBITOR', 'description': 'N...",None,24689268,"[{'comments': None, 'relation': '=', 'result_f...",CHEMBL5105160,Inhibition of His-tagged Mycobacterium tubercu...,B,None,None,BAO_0000190,...,Mycobacterium tuberculosis,Enoyl-[acyl-carrier-protein] reductase,1773,None,None,IC50,uM,UO_0000065,None,0.4
419,"{'action_type': 'INHIBITOR', 'description': 'N...",None,24994756,[],CHEMBL5231331,Inhibition of Mycobacterial tuberculosis ATCC ...,B,None,None,BAO_0000190,...,Mycobacterium tuberculosis,Enoyl-[acyl-carrier-protein] reductase,1773,None,None,IC50,ug ml-1,UO_0000274,None,4.5


## **Data pre-processing of the bioactivity data**

### **Combine the 3 columns (molecule_chembl_id,canonical_smiles,standard_value) and bioactivity_class into a DataFrame**

In [54]:
selection = ['molecule_chembl_id','canonical_smiles','standard_value']
df3 = df2_nr[selection]
df3

,molecule_chembl_id,canonical_smiles,standard_value
0,CHEMBL217926,O=C(Nc1ccccc1)C1CC(=O)N(C2CCCCC2)C1,10660.0
1,CHEMBL216547,O=C(Nc1ccccc1Br)C1CC(=O)N(C2CCCCC2)C1,100000.0
2,CHEMBL213720,O=C(Nc1ccc2c(c1)OCCO2)C1CC(=O)N(C2CCCCC2)C1,100000.0
3,CHEMBL217274,Cc1cccc(C)c1NC(=O)C1CC(=O)N(C2CCCCC2)C1,100000.0
4,CHEMBL217773,O=C(Nc1ccc(Oc2ccccc2)cc1)C1CC(=O)N(C2CCCCC2)C1,100000.0
...,...,...,...
412,CHEMBL4794589,O=[N+]([O-])c1ccc(NC(=S)NCc2ccccc2Br)cc1,15600.0
413,CHEMBL5190515,CCS(=O)(=O)N1N=Cc2ccc(Oc3ccc(C(F)(F)F)cc3/C=N/...,30.0
414,CHEMBL5174930,CS(=O)(=O)N1N=Cc2ccc(Oc3ccc(C(F)(F)F)cc3/C=N/O...,400.0
419,CHEMBL5284077,C=C1C(=O)OC[C@H]1C/C=C(\C)CC(=O)C=C(C)C,4.5


Saves dataframe to CSV file

In [55]:
df3.to_csv('enoyl_acyl_carrier_protein_reductase_02_bioactivity_data_preprocessed.csv', index=False)

### **Labeling compounds as either being active, inactive or intermediate**
The bioactivity data is in the IC50 unit. Compounds having values of less than 1000 nM will be considered to be **active** while those greater than 10,000 nM will be considered to be **inactive**. As for those values in between 1,000 and 10,000 nM will be referred to as **intermediate**. 

In [56]:
df4 = pd.read_csv('enoyl_acyl_carrier_protein_reductase_02_bioactivity_data_preprocessed.csv')

In [57]:
bioactivity_threshold = []
for i in df4.standard_value:
  if float(i) >= 10000:
    bioactivity_threshold.append("inactive")
  elif float(i) <= 1000:
    bioactivity_threshold.append("active")
  else:
    bioactivity_threshold.append("intermediate")

In [58]:
bioactivity_class = pd.Series(bioactivity_threshold, name='class')
df5 = pd.concat([df4, bioactivity_class], axis=1)
df5

,molecule_chembl_id,canonical_smiles,standard_value,class
0,CHEMBL217926,O=C(Nc1ccccc1)C1CC(=O)N(C2CCCCC2)C1,10660.0,inactive
1,CHEMBL216547,O=C(Nc1ccccc1Br)C1CC(=O)N(C2CCCCC2)C1,100000.0,inactive
2,CHEMBL213720,O=C(Nc1ccc2c(c1)OCCO2)C1CC(=O)N(C2CCCCC2)C1,100000.0,inactive
3,CHEMBL217274,Cc1cccc(C)c1NC(=O)C1CC(=O)N(C2CCCCC2)C1,100000.0,inactive
4,CHEMBL217773,O=C(Nc1ccc(Oc2ccccc2)cc1)C1CC(=O)N(C2CCCCC2)C1,100000.0,inactive
...,...,...,...,...
335,CHEMBL4794589,O=[N+]([O-])c1ccc(NC(=S)NCc2ccccc2Br)cc1,15600.0,inactive
336,CHEMBL5190515,CCS(=O)(=O)N1N=Cc2ccc(Oc3ccc(C(F)(F)F)cc3/C=N/...,30.0,active
337,CHEMBL5174930,CS(=O)(=O)N1N=Cc2ccc(Oc3ccc(C(F)(F)F)cc3/C=N/O...,400.0,active
338,CHEMBL5284077,C=C1C(=O)OC[C@H]1C/C=C(\C)CC(=O)C=C(C)C,4.5,active


Saves dataframe to CSV file

In [59]:
df5.to_csv('enoyl_acyl_carrier_protein_reductase_03_bioactivity_data_curated.csv', index=False)

In [60]:
import zipfile
import glob
import os

# Get the Documents folder path
documents_path = os.path.expanduser('~/Documents')
zip_path = os.path.join(documents_path, 'enoyl_acyl_carrier_protein_reductase.zip')

# Create the zip file in Documents folder
with zipfile.ZipFile(zip_path, 'w') as zipf:
    for file in glob.glob('*.csv'):
        zipf.write(file)

---